# Obtención de las posibles ubicaciones de las ambulancias
En este cuaderno de python se implementa la metodología para clasificar los ejes viales que son candidatos para ser puntos de preparación en incidentes con víctimas masivas. Los requisitos previos para ejecutar el cuaderno son:

*   Bibliotecas: geopandas, NetworkX, OSMNX, shapely
*   Archivos: graph_transport.graphml, es el grafo de transporte previamente descargado de OpenStreetMaps en formato graphml

Con este cuaderno se genera el archivo `PLTBs.gpkg` con los resultados de las diferentes etapas de la metodología, separadas en las siguientes capas:

*   `filtro_tipo`: Contiene las vialidades del grafo de transporte filtradas por sus características
*   `filtro_capacidad`: Contiene las vialidades filtradas por tipo ahora filtradas por capacidad
*   `PLTBs`: Contiene las vialidades aptas para ser bases temporales de ambulancias, agrupadas sin sobreposición
*   `PLTBs_grouped`: Contiene las vialidades aptas agrupadas por proximidad

Cada una de estas capas está dividida en aristas y vértices, con los sufijos `_edges` y `_nodes` respectivamente. En las ultimas dos capas, las aristas corresponden a la geometría de las vialidades agrupadas, mientras que los vértices representan el punto de partida de las ambulancias colocadas en cualquier punto de las vialidades en ese grupo.




In [ ]:
# Bibliotecas de uso específico en el cuaderno
from shapely.geometry import MultiLineString, Point
from copy import deepcopy as copy
from collections import deque

# Bibliotecas de uso general en el cuaderno
from myutils import *

## Etapa 1: Selección por características

En esta sección identifican las vialidades candidatas a partir del grafo de transporte según sus características; para esto se siguen los siguientes pasos:

Crea una copia $G_1$ del grafo de transporte $G_T$, del cual se eliminarán las vialidades no aptas 

Para cada arista (u,v) del grafo $G_1$:

Obten el tipo y sentido de la vialidad
*   Si tiene múltiples tipos de vialidad (fue simplificada previamente), toma el tipo de menor jerarquía.
*   Si el tipo de vialidad es una incorporación, una carretera o no está clasificada, elimina la arista (u,v).
*   De lo contrario:
    *   Si se tiene información sobre el número de carriles:
        *   Se toma el número de carriles, o el valor mínimo en caso de contar con múltiples valores.
        *   Elimina la arista (u,v) si tiene menos de 3 carriles ó tiene menos de 4 pero es residencial o de doble sentido.
    *   Si no se tiene:
        *   Elimina la arista si es de tipo residencial o terciaria.



In [ ]:
# Carga el grafo de la red de transporte de la Ciudad de México
g = ox.load_graphml(filepath="../GeoData/graph_transport.graphml")
print('Grafo de transporte cargado')
output_file = r'../GeoData/PLTBs_test.gpkg'

In [ ]:
# Crea una copia para filtrar el grafo
g1 = g.copy()

# descarta viaildades de tipo unclassified, residential y motorway
discard = set(['unclassified','motorway'])
# jerarquía de vialidades
hierarchy = {'unclasified':0, 'residential':1, 'tertiary_link':2, 'tertiary':3, 'secondary_link':4, 'secondary':5, 'primary_link':6, 'primary':7, 'trunk_link':8, 'trunk':9, 'motorway_link':10, 'motorway':11}


# Elimina vialidades con menos de 3 carriles y que sean incorporaciones
for u, v, k, data in g.edges(keys=True, data=True):
        
    highway = data.get('highway', '')
    oneway = data.get('oneway', False)
    
    # Si highway es una lista, se toma el valor mínimo de la lista
    if type(highway) == list:
        highway = min(highway, key=lambda x: hierarchy.get(x, 0))
    
    # Si el tipo de vialidad es un link (incorporación) o es una vialidad que se desea descartar, se elimina la arista
    if 'link' in highway or highway in discard:
        g1.remove_edge(u, v, k)
    
    # De lo contrario, se revisa el número de carriles
    else:
        
        # Si se tiene el número de carriles, se toma directamente
        if 'lanes' in data:
            
            lanes = data.get('lanes')
            
            # Si lanes es una lista, se toma el valor mínimo de la lista
            if type(lanes) == list:
                lanes = min(lanes)   
            
            # Si el número de carriles es menor a 3, o si es residencial o de doble sentido y son menos de 4 carriles, se elimina la arista
            if lanes < '3' or ( lanes < '4' and (oneway == False  or highway == 'residential' ) ) :
                g1.remove_edge(u, v, k)
        
        # Si no se tiene el número de carriles, se toma el tipo de vialidad
        else:
            
            if highway == 'tertiary' or highway == 'residential':
                g1.remove_edge(u, v, k)
        
print('Grafo filtrado por número de carriles y vialidad')
print(len(list(g1.nodes())))

In [ ]:
# Elimina nodos aislados
isolated_nodes = list(nx.isolates(g1))
g1.remove_nodes_from(isolated_nodes)

print('Grafo simplificado por tipo de vialidad y número de carriles')
print(len(list(g1.nodes())))

In [ ]:
fig, ax = ox.plot_graph(g,
                        edge_linewidth = 0.2, 
                        node_color = (0,0,0,0),
                        node_size = 0,
                        dpi = 72,
                        figsize=(16, 16),
                        show=False)

ox.plot_graph(g1,
                edge_linewidth = 0.2, 
                edge_color = (0,0.5,0.9,1),
                node_color = (0,0.5,0.9,1),
                node_size = 1,
                dpi = 72,
                figsize=(16, 16),
                ax=ax)

fig.show()

In [ ]:
# Guarda el grafo filtrado en un archivo .geopackage
save_graph_geopackage(g1, filepath=output_file, layer = 'filter_features', directed=True)

## Etapa 2: Selección de vialidades por capacidad

Posteriormente se seleccionan únicamente las vialidades que tienen una capacidad para almacenar más de `$n$` ambulancias (un hiper parámetro que define el usuario). La razón por la cual se implementa este procedimiento en una etapa diferente, es que el grafo resultante anterior contiene vialidades "transitables" por las ambulancias, y estas vialidades se pueden considerar para agrupar los puntos en etapas posteriores, es decir, aunque no se puedan usar como una base temporal, sí pueden ser transitadas fácilmente por las ambulancias para llegar a otra vialidad que sí es candidata, lo que resulta de utilidad para el proceso de agrupación por proximidad. 

Los hiper parámetros que se deben definir para esta etapa son:
*   `ĺength_ambulance`: es la longitud promedio de una ambulancia, en este caso se consideran 6 metros.
*   `min_length`: es la cantidad mínima de ambulancias que debe ser capaz de contener una base temporal.

In [ ]:
# Parámetros para la reducción
length_ambulance = 6 # metros

# Elimina aristas con poca capacidad
minimum_capacity = 15 # mínima cantidad de vehículos
min_length = minimum_capacity* length_ambulance # longitud mínima de la arista


g2 = g1.copy()

edges_to_remove = []
for u, v, k, data in g2.edges(keys=True, data=True):
    if data.get('length', 0) < min_length:
        edges_to_remove.append((u, v, k))

g2.remove_edges_from(edges_to_remove)

# Elimina nodos aislados
isolated_nodes = list(nx.isolates(g2))
g2.remove_nodes_from(isolated_nodes)
print(len(list(g2.nodes())))
save_graph_geopackage(g2, filepath=output_file, layer = 'filter_capacity', directed=True)


### Obtiene los nodos finales de las vialidades resultantes

In [ ]:
registers = {}
registered = set()

# Función para asegurar que un dato sea una lista
enlist = lambda x: x if type(x) == list else [x]

for u, v, k, data in g2.edges(keys=True, data=True):
    v_point = g2.nodes[v]
    u_point = g2.nodes[u]
    segments = data['osmid']

    nombre = enlist( data['name'] if 'name' in data else 'Sin nombre')
    vialidad = enlist( data['highway'] if 'highway' in data else 'unclassified')
    sentido = enlist( data['oneway'] if 'oneway' in data else None)
    
    largo = data['length'] if 'length' in data else 0
    capacidad = data['length']/length_ambulance if 'length' in data else 0

    geom_point = [v_point['x'], v_point['y']]
    
    if 'geometry' in data:
        geom_line = [i for i in data['geometry'].coords]

    else:
        geom_line = [(u_point['x'], u_point['y']), (v_point['x'], v_point['y'])]
    
    
    new_reg = { 'y' : v_point['y'],
                'x': v_point['x'],
                'lon': v_point['lon'],
                'lat': v_point['lat'],
                'node': v,
                'merged': 1,
                't_length': largo,
                't_capacity': capacidad,
                'length': [largo],
                'capacity': [capacidad],
                'streets': nombre,
                'oneway': sentido,
                'grouped': [u],
                # 'geom_point' : "POINT ("+str(point_geo[0])+" "+str(point_geo[1])+")"}
                'geom_point' : geom_point,
                'geom_line' : [geom_line] }
    
    if v in registered:
        
        previous = registers[v]

        # Numeric attributes
        new_reg['merged'] = previous['merged'] + 1
        new_reg['t_length'] = previous['t_length'] + new_reg['t_length']
        new_reg['t_capacity'] = previous['t_capacity'] + new_reg['t_capacity']     
        
        # Other attributes
        new_reg['streets'] = previous['streets'] + new_reg['streets']      
        new_reg['length'] = previous['length'] + new_reg['length']
        new_reg['capacity'] = previous['capacity'] + new_reg['capacity']
        new_reg['oneway'] = previous['oneway'] + new_reg['oneway']
        new_reg['grouped'] = previous['grouped'] + new_reg['grouped']
        new_reg['geom_line'] = previous['geom_line'] + new_reg['geom_line']
        
    registers[v] = new_reg
    registered.add(v)

In [ ]:
# Crea un dataframe con los registros
df = pd.DataFrame.from_records([i for i in registers.values()],index='node')
df['geom_point'] = df['geom_point'].apply(lambda x: Point(x[0],x[1]))
df['geom_line'] = df['geom_line'].apply(lambda x: MultiLineString(x))

# Serializa los datos de tipo lista
df['streets'] = df['streets'].apply(lambda x: json.dumps(x, ensure_ascii=False))
df['oneway'] = df['oneway'].apply(json.dumps)
df['length'] = df['length'].apply(json.dumps)
df['capacity'] = df['capacity'].apply(json.dumps)
df['grouped'] = df['grouped'].apply(json.dumps)

# Almacena los datos en un archivo geopackage
bases_puntos = gpd.GeoDataFrame(df.loc[:, df.columns != 'geom_line'], crs="EPSG:6369", geometry = 'geom_point')
bases_puntos.to_file(output_file, layer='filter_overlaps_nodes', driver="GPKG", encoding='utf-8')

bases_lineas = gpd.GeoDataFrame(df.loc[:, df.columns != 'geom_point'], crs="EPSG:6369", geometry = 'geom_line')
bases_lineas.to_file(output_file, layer='filter_overlaps_edges', driver="GPKG", encoding='utf-8')

print(f'{len(registers)} vialidades registradas sin sobreposición')

## Etapa 3: Agrupamiento por proximidad

En esta etapa se agrupan las bases obtenidas de la etapa anterior según un grado de proximidad, para ello se definen los hiperparámetros:

*   `minimum_speed`: es la velocidad mínima a la que una ambulancia podría transitar de una base a otra.
*   `connect_time`: es el tiempo máximo permitido para trasladarse de una base a otra.

El algoritmo al final agrupa y selecciona los nodos representativos de los grupos de forma que garantiza que podría llegar desde éste nodo representativo hasta cualquier otro nodo del mismo grupo en menos de `$t$` minutos, por lo cual se pueden usar los nodos representativos como PLTBs.

Otra forma de usar este algoritmo es con el grafo inverso, en el que los grupos formados representan a las vialidades de las cuales se puede llegar al nodo representativo en menos de `$t$` minutos. En esta implementación se optó por esta opción.

In [ ]:
# Definición de parámetros para la agrupación de vialidades
minimum_speed = 20 # velocidad mínima en km/h
connect_time = 2 # tiempo de conexión en minutos
connect_distance = round((20*1000)/60 * connect_time) # distancia de conexión en metros

In [ ]:
# Función para recorrer el grafo con dfs

def dfs_reverse(graph, set_nodes, start_node, lmax):
    visited = set()
    stack = [(start_node, 0, 0)]  # Tupla de (nodo, profundidad)
    depth_dict = {}  # Diccionario para almacenar la profundidad de cada nodo

    while stack:
        node, depth, length = stack.pop()

        if node not in visited :
            visited.add(node)
            
            if node in set_nodes:
                depth_dict[node] = depth  # Almacenar la profundidad del nodo
                
            neighbors = list(graph.predecessors(node))  # Obtener vecinos en dirección contraria
            
            for neighbor in neighbors:
                
                # data = graph.get_edge_data(*(node,neighbor, 0)) # Cuando todas las aristas tienen una sola llave que es 0
                data = list(graph.get_edge_data(neighbor,node).values())[0] # Cuando las aristas tienen más de una llave
                edge_length = data['length']
                
                # si está a una distancia menor:
                if length + edge_length < lmax : 
                    
                    if neighbor not in visited:
                        stack.append((neighbor, depth + 1, length + edge_length)) # Incrementar la profundidad en cada nivel

    return depth_dict


# Función para recorrer el grafo con bfs
def bfs(graph_transport, suitable_nodes, start, lmax):
    
    # Nodos visitados
    visited = set()
    
    # Grupos creados
    groups = []
    grouped = []
    total_nodes_grouped = 0
    
    # Nodos que no han sido agrupados
    others = deque([start]) 
    
    # Mientras haya nodos que no han sido agrupados
    while others:
        
        # Tomar el primer nodo de la lista de nodos no agrupados
        head = others.popleft()
        
        # Si el nodo no ha sido visitado
        if head not in visited:
            
            # Agregar el nodo a los visitados
            visited.add(head)
            
            # Crear una cola del grupo con el nodo como cabeza
            queue = deque([ [head, 0] ])

            # Crear un grupo con el nodo como cabeza
            group = [head]
            
            # Mientras haya nodos en la cola de ese grupo
            while queue:
                
                # Tomar el primer nodo de la cola, su grado y su longitud acumulada
                node, cum_len = queue.popleft()
                # node_degree = graph_transport.nodes[node]['street_count']
                node_degree = len(set(list(graph_transport.successors(node))).intersection(suitable_nodes))
                
                # Obtener los vecinos del nodo
                neighbors = list(graph_transport.successors(node))
                
                # Para cada vecino
                for neighbor in neighbors:
                    
                    # Obtener la longitud de la arista
                    # data = graph.get_edge_data(*(node,neighbor, 0)) # Cuando todas las aristas tienen una sola llave que es 0
                    data = list(graph_transport.get_edge_data(node,neighbor).values())[0] # Cuando las aristas tienen más de una llave
                    edge_length = data['length']
                    
                    # Grado del vecino considerando solo vecinos candidatos
                    # neighbor_degree = len(set(list(graph_transport.successors(neighbor))).intersection(suitable_nodes))
                    neighbor_degree = len(set(list(graph_transport.successors(neighbor))).intersection(suitable_nodes))

                    # Si el vecino no ha sido visitado
                    if (neighbor not in visited):
                        
                        # Si la longitud acumulada es menor a la longitud máxima y una de dos:
                        # o el vecino no es candidato, o sí es candidato pero su grado es menor o igual al grado del nodo anterior
                        if(cum_len + edge_length < lmax) and ( (neighbor_degree <= node_degree and neighbor in suitable_nodes)  or neighbor not in suitable_nodes):
                            
                            # Agregar el vecino a la cola del grupo actual
                            queue.append([neighbor, cum_len + edge_length])
                            
                            # Agregar el vecino al grupo actual si es candidato
                            if neighbor in suitable_nodes:
                                group.append(neighbor)
                            
                            # Agregar el vecino a los visitados
                            visited.add(neighbor)
                            
                            
                        # Si no se cumple alguna de las condiciones anteriores, no se puede agrupar en ese grupo
                        else:
                            # Agregar el vecino a la lista de nodos no agrupados si es candidato
                            if neighbor in suitable_nodes:
                                others.append(neighbor)
                            
            # Agregar el grupo a la lista de grupos
            grouped += group
            groups.append(copy(group))
            total_nodes_grouped += len(group)
    
    # Regresar los grupos y los nodos agrupados
    return groups, grouped

In [ ]:
# Inicializar variables
suitable_nodes = set(list(registers.keys()))
remaining_nodes = set(list(registers.keys()))
subgraphs = []
inversed_graph = g1.reverse()

# Mientras haya nodos que no han sido agrupados
while remaining_nodes:
    
    # Identifica nodos aún disponibles
    suitable_nodes=copy(remaining_nodes)
    
    # Selecciona un nodo para comenzar el algoritmo
    current_node = remaining_nodes.pop()

    # Realizar el recorrido DFS inverso y obtener la profundidad de cada vecino
    predecessors_nodes = dfs_reverse(inversed_graph, copy(suitable_nodes), copy(current_node), connect_distance)
    predecessors_list = list(predecessors_nodes)

    # Encuentra el vecino más lejano que puede llegar al nodo
    farthest_predecessor = copy(max(predecessors_list, key=lambda x: predecessors_nodes[x]))
    
    # Realizar el recorrido BFS y obtener los grupos 
    groups, grouped = bfs(inversed_graph, copy(suitable_nodes), farthest_predecessor, connect_distance)
    
    
    # Actualizar los nodos disponibles
    remaining_nodes = remaining_nodes.difference(grouped)
    
    # Agregar los grupos a la lista de grupos
    subgraphs += groups

print('total de grupos: ',len(subgraphs))
print('total de agrupados: ', sum([len(i) for i in subgraphs]))
print('Disponibles originales: ', len(registers))
print('faltantes de grupo: ',len(registers) - sum([len(i) for i in subgraphs]))

In [ ]:
# Crea los grupos en un dataframe
# Copia los registros ya obtenidos
registers_grouped = copy(dict(registers))

# Por cada grupo
for group in subgraphs:
    
    # Si el grupo tiene más de un elemento
    if len(group) > 1:
        
        # Selecciona el primer nodo como nodo principal
        head_node = group[0]

        # Por cada nodo en el grupo
        for node in group[1::]:
            
            # Si el nodo ya está en los registros, se fusionan los datos
            if node in registers_grouped:
                main_reg = dict(registers_grouped[head_node])
                new_reg = dict(registers_grouped[node])

                # Atributos numéricos
                main_reg['merged'] = main_reg['merged'] + new_reg['merged']
                main_reg['t_length'] = main_reg['t_length'] + new_reg['t_length']
                main_reg['t_capacity'] = main_reg['t_capacity']  + new_reg['t_capacity']           
                
                # Atributos no numéricos
                main_reg['streets'] = main_reg['streets'] + new_reg['streets']
                main_reg['length'] = main_reg['length'] + new_reg['length']
                main_reg['capacity'] = main_reg['capacity'] + new_reg['capacity']
                main_reg['oneway'] = main_reg['oneway'] + new_reg['oneway']
                main_reg['grouped'] = main_reg['grouped'] + new_reg['grouped']
                main_reg['geom_line'] = main_reg['geom_line'] + new_reg['geom_line']

                # Se elimina el nodo de los registros
                registers_grouped.pop(node)   
                
                # Se actualiza el nodo principal
                registers_grouped[head_node] = main_reg
  
            else:
                print('Error in node :', node)
        
# Crea un dataframe con los registros agrupados
df = pd.DataFrame.from_records([i for i in registers_grouped.values()],index='node')
df['geom_point'] = df['geom_point'].apply(lambda x: Point(x[0],x[1]))
df['geom_line'] = df['geom_line'].apply(lambda x: MultiLineString(x))

# Serializa los datos de tipo lista
df['streets'] = df['streets'].apply(lambda x: json.dumps(x, ensure_ascii=False))
df['oneway'] = df['oneway'].apply(json.dumps)
df['length'] = df['length'].apply(json.dumps)
df['capacity'] = df['capacity'].apply(json.dumps)
df['grouped'] = df['grouped'].apply(json.dumps)

# Almacena los datos en un archivo geopackage
bases_puntos = gpd.GeoDataFrame(df.loc[:, df.columns != 'geom_line'], crs="EPSG:6369", geometry = 'geom_point')
bases_puntos.to_file(output_file, layer='PLTBs_nodes', driver="GPKG", encoding="utf-8")

bases_lineas = gpd.GeoDataFrame(df.loc[:, df.columns != 'geom_point'], crs="EPSG:6369", geometry = 'geom_line')
bases_lineas.to_file(output_file, layer='PLTBs_edges', driver="GPKG", encoding="utf-8")

print(len(registers_grouped),' PLTBs obtenidos')
